Following P. Olofsson et al.:

(page 52 sect. 5)

This accuracy asessment is designed for the objectives of:
- estimating overall and class-specific accuracies
- areas of the individual classes (as determined by the reference classification), and
- confidence intervals for each accuracy and area parameter


This sampling design follows a stratified random design per ap class (iceplant, other vegetation, low ndvi, water).

In [1]:
import os
import numpy as np
import pandas as pd

### Pixels per class in full map of SB coast

In [2]:
year = 2020
prefix = 'modelAE5_FP_2020'
pixel_count = pd.read_csv(os.path.join(os.getcwd(), prefix+'_rasters_'+str(year)+'_pixel_counts.csv'))
pixel_count

,n_nonice_2020,n_ice_2020,n_ground_2020,n_water_2020,raster
0,36271293,5382187,111150412,62968690,modelAE5_FP_2020_merged_crs26910_S_2020
1,1122203,30004,1891593,2893071,modelAE5_FP_2020_merged_crs26910_W_2020
2,89669636,1123921,62587031,69125241,modelAE5_FP_2020_merged_crs26911_2020


In [3]:
# total number of pixels in neach category
n_pix = [sum(pixel_count.iloc[:,c]) for c in range(0,4)]
n_pix

[127063132, 6536112, 175629036, 134987002]

### 5.1.1  Determining the sample size

For stratified random sampling Cochran (1977 eq 5.25) suggests the following sample size formula:

$n \approx \left( \frac{\sum W_iS_i}{S(\hat{O})}\right)^2$, where

$S(\hat{O})$ is the standard error of the estimated overall accuracy that we would like to achieve

$W_i$ is the fraction of the map labeled as class $i$

$S_i = \sqrt{U_i(1-U_i)}$ is the standard deviation of class $i$.


In this last formula, $U_i$ are the user's accuracies for class $i$, i.e. the precision for each class: TP/(TP+FP).

Recall that the square root of the estimated variance results in the standard error of the estimator. For example, in the case of the estimated overall accuracy of the map $\hat{O}$ we have that $\hat{S}(\hat{O}) = \sqrt{\hat{V}(\hat{O})}$ (see eq. 5). Also, the standard error is used to get confidence intervals for the estimated statistic:the 95% confidence interval is estimated as $\hat{O} \pm 1.96 \hat{S}(\hat{O}) = \hat{O} \pm 1.96 \sqrt{\hat{V}(\hat{O})}$.

For determining the sample size then we have to estimate the user's accuracies $U_i$ and the std error of the estimated overall accuracy $S(\hat{O})$. The class fractions $W_i$ are not estimated since they come from counting the number of pixels in each class in the map. 

To estimate $U_i$ (and thus obtain the standard deviations) we can use the precisions obtained from applying the model on the test set. Take into account $U_i$ from the model testing might still be optimistic. For low ndvi and water we use 0.95 and 0.9 respectively.

To find a value of $S(\hat{O})$ we feel comfortable with, we need to take into account the following: suppose we want an overall accuracy ((TP + TN)/(P+N)) of 0.9, meaning 90% of the pixels were correctly classified. A std error of 0.01 would mean that the confidence interval around this estimated overall accuracy is $0.9 \pm 0.01*1.96 = 0.9 \pm 0.0196$, so OA is betwee 88% and 92%. If we increase std error to 0.015, OA would be (approx) between 87% and 93%. 



In [6]:
# Recreation of sampling design by SEPAL

# ---------------------------------------------
# --------------- PARAMETERS ------------------
# standard error for all the points
std_error = 0.015

# user's accuracies TP/(TP+FP) (estimates)
# classes are: [other vegetation, iceplant, low ndvi, water]
U = [0.87, 0.89, 0.9, 0.95]
# ---------------------------------------------

# fraction of pixels with a given class in total pixels
total_pix = sum(n_pix)
pix_prop = [n/total_pix for n in n_pix]

# standard deviation of user's accuracies
stdv = [ np.sqrt(u*(1-u)) for u in U]

X = [ x*y for x,y in zip(pix_prop, stdv)]

sample_size = (sum(X)/std_error)**2
sample_size

# distributing sample size among classes
#[...]

363.07223806454977

In [7]:
errors_U =[]
strat_samples = []
strat_title = []

def strat_stderror(U, strat_sample):
    return [ np.sqrt(u*(1-u)/(n-1)) for u,n in zip(U, strat_sample) ]

In [8]:
# Distributing sample size among classes

# ---------------------------------------------
strat_title.append('equal')
sample_equal = [sample_size/4 for i in range(0,4)]
strat_samples.append(sample_equal)
errors_U.append(strat_stderror(U, sample_equal))

# ---------------------------------------------
class2_prop = n_pix[2]/(n_pix[2]+n_pix[3])
class3_prop = n_pix[3]/(n_pix[2]+n_pix[3])

# ---------------------------------------------
strat_title.append('120')
sample_120 = [120, 120, (sample_size-240)*class2_prop, (sample_size-240)*class3_prop] 
strat_samples.append(sample_120)
errors_U.append(strat_stderror(U, sample_120))

# ---------------------------------------------
strat_title.append('100')
sample_100 = [100, 100, (sample_size-200)*class2_prop, (sample_size-200)*class3_prop] 
strat_samples.append(sample_100)
errors_U.append(strat_stderror(U, sample_100))

# ---------------------------------------------
strat_title.append('87')
sample_87 = [87, 87, (sample_size-174)*class2_prop, (sample_size-174)*class3_prop] 
strat_samples.append(sample_87)
errors_U.append(strat_stderror(U, sample_87))

# ---------------------------------------------
strat_title.append('75')
sample_75 = [75, 75, (sample_size-150)*class2_prop, (sample_size-150)*class3_prop] 
strat_samples.append(sample_75)
errors_U.append(strat_stderror(U, sample_75))

# ---------------------------------------------
strat_title.append('prop')
sample_prop = [sample_size*x for x in pix_prop]
strat_samples.append(sample_prop)
errors_U.append(strat_stderror(U, sample_prop))


In [9]:
strat_df = pd.DataFrame(strat_samples).T
strat_df.columns  = strat_title
strat_df

,equal,120,100,87,75,prop
0,90.76806,120.000000,100.000000,87.000000,75.000000,103.853014
1,90.76806,120.000000,100.000000,87.000000,75.000000,5.342186
2,90.76806,69.587709,92.204576,106.905539,120.475659,143.547576
3,90.76806,53.484529,70.867662,82.166699,92.596579,110.329462


In [10]:
errors_df = pd.DataFrame(errors_U).T
errors_df.columns = strat_title
errors_df

,equal,120,100,87,75,prop
0,0.035495,0.030829,0.033800,0.036265,0.039094,0.033161
1,0.034298,0.029789,0.032660,0.035042,0.037776,0.155947
2,0.031664,0.036224,0.031413,0.029152,0.027446,0.025127
3,0.023003,0.030084,0.026074,0.024191,0.022772,0.020844


In [11]:
196*errors_df

,equal,120,100,87,75,prop
0,6.957071,6.042462,6.624754,7.107848,7.662518,6.499484
1,6.722443,5.838678,6.401333,6.868135,7.404098,30.565662
2,6.206066,7.099932,6.156998,5.713707,5.379447,4.924900
3,4.508603,5.896404,5.110516,4.741480,4.463374,4.085397


In [25]:
100 + 100 + 150 + 115
# GOING FOR THE 100 PER MINORITY CLASS

465

In [ ]:
100 + 100 + 150 + 100